In [4]:
import sys
!{sys.executable} -m pip install syft

     |████████████████████████████████| 433 kB 3.2 MB/s eta 0:00:01
     |████████████████████████████████| 177 kB 57.6 MB/s eta 0:00:01
     |████████████████████████████████| 94 kB 7.6 MB/s  eta 0:00:01
     |████████████████████████████████| 484 kB 53.5 MB/s eta 0:00:01
     |████████████████████████████████| 2.0 MB 31.7 MB/s eta 0:00:01
     |███████████████████             | 446.7 MB 55.2 MB/s eta 0:00:064

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



     |███████████████████████████████▍| 739.8 MB 104.6 MB/s eta 0:00:01

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



     |████████████████████████████████| 753.4 MB 59 kB/s 
     |████████████████████████████████| 54 kB 6.7 MB/s  eta 0:00:01
     |████████████████████████████████| 20.6 MB 60.8 MB/s eta 0:00:01
     |████████████████████████████████| 449 kB 74.8 MB/s eta 0:00:01
     |████████████████████████████████| 4.0 MB 35.4 MB/s eta 0:00:01
     |████████████████████████████████| 73 kB 6.0 MB/s  eta 0:00:01
     |████████████████████████████████| 57 kB 9.6 MB/s  eta 0:00:01
     |████████████████████████████████| 78 kB 13.5 MB/s eta 0:00:01
     |████████████████████████████████| 200 kB 65.2 MB/s eta 0:00:01
     |████████████████████████████████| 303 kB 37.8 MB/s eta 0:00:01
     |████████████████████████████████| 63 kB 8.9 MB/s  eta 0:00:01
     |████████████████████████████████| 26.0 MB 66.7 MB/s eta 0:00:01
     |████████████████████████████████| 1.8 MB 33.9 MB/s eta 0:00:01
     |████████████████████████████████| 9.0 MB 65.2 MB/s eta 0:00:01
     |████████████████████████████████| 126 kB 3

# Ensure Proper Downloading

In [2]:
import sys

import torch
from torch.nn import Parameter
import torch.nn as nn
import torch.nn.functional as F

import syft as sy
hook = sy.TorchHook(torch)

torch.tensor([1,2,3,4,5])

tensor([1, 2, 3, 4, 5])

## Basic Tools

In [27]:
# How tensors work together
x = torch.tensor([3, 4, 5])
y = x*2 - 3
print (y)

# work with pointers to tensors, create "pretend" machine

alice_machine = sy.VirtualWorker(hook, id="alice")

# fake data to send to alice
fake1 = torch.tensor([3, 8, 18])
fake2 = torch.tensor([1, 4, 39])
print (fake1 + fake2)

# this sends these tensors to Alice's machine
fake1_ptr = fake1.send(alice_machine)
fake2_ptr = fake2.send(alice_machine)

print ("look below at the pointer: \n" + str(fake1_ptr))
# check out out alice's tensors
alice_machine._objects

tensor([3, 5, 7])
tensor([ 4, 12, 57])
look below at the pointer: 
(Wrapper)>[PointerTensor | me:73588783925 -> alice:94082622159]


{59826220630: tensor([ 4, 12, 57]),
 32371516483: tensor([ 4, 12, 57]),
 94082622159: tensor([ 3,  8, 18]),
 18129534882: tensor([ 1,  4, 39])}

## Playing further around with this concept

In [57]:
# add the sum of the first two tensors
fake3 = fake1_ptr + fake2_ptr
alice_machine._objects



{59826220630: tensor([ 4, 12, 57])}

In [58]:
fake1_ptr.location

In [59]:
alice_machine

<VirtualWorker id:alice #objects:1>

In [42]:
# alice_machine, location, reference to location that pointer pointing to
alice_machine == fake1_ptr.location

True

In [43]:
# id where tensor stored at location
fake1_ptr.id_at_location

94082622159

In [39]:
# can see which worker owns the pointer tensors, here it is the local worker ("me")
fake1_ptr.owner

<VirtualWorker id:me #objects:0>

In [44]:
# by default have VirtualWorker for us, automtically created by hook = sy.TorchHook()
me = sy.local_worker
me

<VirtualWorker id:me #objects:0>

In [41]:
me == fake1_ptr.owner

True

In [45]:
fake1_ptr.get()

tensor([ 3,  8, 18])

In [46]:
fake2_ptr.get()

tensor([ 1,  4, 39])

In [47]:
fake3.get()

tensor([ 4, 12, 57])

In [60]:
alice_machine._objects

{59826220630: tensor([ 4, 12, 57])}

### Using Tensor Pointers

In [68]:
alice3 = sy.VirtualWorker(hook, id="alice3")
x = torch.tensor([1, 2, 4, 8]).send(alice3)
y = torch.tensor([3, 5, 7, 11]).send(alice3)
alice3

<VirtualWorker id:alice3 #objects:2>

In [73]:
# observe how this automatically gets sent to machine
z = x + y

In [70]:
z

(Wrapper)>[PointerTensor | me:90707844158 -> alice3:52995406229]

In [71]:
z.get()

tensor([ 4,  7, 11, 19])

In [72]:
alice3

<VirtualWorker id:alice3 #objects:2>